# 7b. Generate edges that connect walk network to public transport stops

In [ ]:
# Standard library and path imports
import set_path

# Third-party library imports
import geopandas as gpd
import momepy
import networkx as nx
import folium

# Local or project-specific imports
import plot_utils
import crossing_utils
import settings as st

if st.my_run == "azure":
    import config_azure as cf
elif st.my_run == "local":
    import config as cf

## Import walking network 

In [ ]:
# Get basic pedestrian network with widths
gdf_network = gpd.read_file(cf.output_file_widths).to_crs(crs=st.CRS)
gdf_network_nodes = gpd.GeoDataFrame(geometry = gdf_network.boundary.explode(index_parts=True), crs=st.CRS)
gdf_network_nodes['x'], gdf_network_nodes['y'] = gdf_network_nodes.geometry.x, gdf_network_nodes.geometry.y
gdf_network_nodes.reset_index()
graph_network = momepy.gdf_to_nx(gdf_network, approach='primal')

In [ ]:
# Calculate connected components for pedestrian network
cc_network = list(nx.connected_components(graph_network))
nodes_in_edge = [line.coords[0] for line in gdf_network['geometry'].to_list()]
cc_ids = [i for node in nodes_in_edge for i in range(len(cc_network)) if node in cc_network[i]]
gdf_network_nodes['cc'] = [x for y in zip(cc_ids, cc_ids) for x in y]

##  Generate edges to include public transport stop features

### Import public transport stop features

In [ ]:
# # Import public transport stop features
gdf_ptf = gpd.read_file(cf.output_public_transport_features).to_crs(crs=st.CRS)

### Generate edges to connect public transport stop nodes

In [ ]:
# Get coordinates of public transport nodes
gdf_ptf['x'], gdf_ptf['y'] = gdf_ptf.geometry.x, gdf_ptf.geometry.y

# Generate edges
gdf_network_edges = crossing_utils.get_connections(gdf_ptf, gdf_network_nodes, max_dist=15, max_connections=1, crs=st.CRS, include_cc_rule=True, cc_column='cc')
print('Number of new edges from osm node to network node:', len(gdf_network_edges))

In [ ]:
# Assign obstacle_free_width_float to connection edges
touching_indices = [gdf_network_edges[gdf_network_edges.geometry.touches(row['geometry'])].index.to_list() for _, row in gdf_ptf.iterrows()]
for i in range(len(touching_indices)):
    gdf_network_edges.loc[touching_indices[i], 'obstacle_free_width_float'] = gdf_ptf.iloc[i]['obstacle_free_width_float']

## Store

In [ ]:
gdf_network_edges.to_file(cf.output_walk_public_transport_stop_connections, driver='GPKG')

## Visualize connection edges

In [ ]:
# set True for satellite background, False for standard background
satellite = False

# Create Folium map
map = folium.Map(
    location=[52.350547922223434, 4.7940192423718443], tiles=plot_utils.generate_map_params(satellite=satellite),
    min_zoom=10, max_zoom=25, zoom_start=17,
    zoom_control=True, control_scale=True, control=False
    )

# Add network and new edges
geo_j = folium.GeoJson(gdf_network, style_function=lambda x: {"color": "black", "weight": 2}).add_to(map)
feature_names = gdf_network_edges.columns.tolist()
feature_names.remove('geometry')
geo_j = folium.GeoJson(gdf_network_edges, tooltip=plot_utils.gen_tooltip(feature_names, feature_names), style_function=lambda x: {"color": "green", "weight": 6}).add_to(map)

# Add crossing features
feature_names = gdf_ptf.columns.tolist()
feature_names.remove('geometry')
color_column = 'wheelchair_accessible'
geo_j = folium.GeoJson(gdf_ptf, tooltip=plot_utils.gen_tooltip(feature_names, feature_names), 
                       marker=folium.Marker(icon=folium.Icon(icon='solid fa-code-merge', prefix='fa')), 
                       style_function =lambda feature: {"markerColor": 'green' if feature["properties"][color_column] == 'Yes' else 
                       'red' if feature["properties"][color_column] == 'No' else
                       'orange'}).add_to(map)
map